In [1]:
# SK KoBERT에서 요구하는 패키지
# boto3 <=1.15.18
# gluonnlp >= 0.6.0, <=0.10.0
# mxnet >= 1.4.0, <=1.7.0.post2
# onnxruntime == 1.8.0, <=1.8.0
# sentencepiece >= 0.1.6, <=0.1.96
# torch >= 1.7.0, <=1.10.1
# transformers >= 4.8.1, <=4.8.1

In [2]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [3]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

# torch, nlp를 위한 모듈 로드
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np

from datetime import datetime
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import re
import gc
import os

# kobert 모듈 로드
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# transformer 모듈 로드
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

# argument
import argparse

#GPU 사용 시
device = torch.device("cuda:0")
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"  # Set the GPU 2 to use

In [4]:
data = pd.read_csv('./train.csv')[['overview', 'cat3']]

In [5]:
data.shape

(16986, 2)

In [6]:
data.head()

,overview,cat3
0,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,항구/포구
1,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,골프
2,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,한식
3,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,한식
4,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,한식


In [7]:
new_overview = []
for i in data.iterrows():
    add_text = re.sub('[^0-9가-힣MTBATV]',' ',i[1]['cat3'])
    new_overview.append(add_text + '에 대한 내용입니다. '+ i[1]['overview'])
data['overview'] = new_overview

In [8]:
data.head(3)

,overview,cat3
0,항구 포구에 대한 내용입니다. 소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이...,항구/포구
1,골프에 대한 내용입니다. 경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다....,골프
2,한식에 대한 내용입니다. 금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는...,한식


In [9]:
data_list = []
for ques, label in zip(data['overview'], data['cat3']):
    tmp = []   
    tmp.append(ques)
    tmp.append(str(label))

    data_list.append(tmp)

In [10]:
data_list[0]

['항구 포구에 대한 내용입니다. 소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 올리고 있으며 바다낚시터로도 유명하다. 항 주변에 설치된 양식장들은 섬사람들의 부지런한 생활상을 고스 란히 담고 있으며 일몰 때 섬의 정경은 바다의 아름다움을 그대로 품고 있는 듯하다. 또한, 섬에는 각시여 전설, 도둑바위 등의 설화가 전해 내려오고 있으며, 매년 정월 풍어제 풍속이 이어지고 있다.<br>',
 '항구/포구']

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [12]:
# kobert 모듈 로드
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [13]:


# 학습 데이터셋 구축 클래스
class BERTDataset(Dataset):
    
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

    
# BERT 분류기 클래스
class BERTClassifier(nn.Module):
    
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 128, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
#         print(attention_mask)
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out), pooler    
    

# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [14]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()

using cached model. /home/rok/Multimodal_Clf/Multimodal-Clf/.cache/kobert_v1.zip
using cached model. /home/rok/Multimodal_Clf/Multimodal-Clf/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/rok/Multimodal_Clf/Multimodal-Clf/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [15]:
device = torch.device("cuda:0") #GPU 사용 시
#device = torch.device("cpu") #CPU 사용 시
torch.cuda.is_available()

# 토크나이저 생성
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

model = BERTClassifier(bertmodel, dr_rate=0.2).to(device)
# model = nn.DataParallel(model, device_ids=[0,1])
model.cuda()
model = nn.DataParallel(model).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [16]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
data['cat3']= label_encoder.fit_transform(data['cat3'])
data['cat3'].unique()

array([120,   8, 118,  73,  58,  90,  85,  91,  94,  97,  12,  31,  67,
        41, 101, 119, 105,  59, 100,  72,  96,  86, 121,   0,  37,  11,
        54,  44,  25,   9,  20,  48, 115,  53,  16,  15,  92,  95, 127,
        82,  61,  84,  52,   3,  93,  43,  35,   5, 116,  57,  99,  27,
        19,   7,  13,   4,  71,  79,  34,  29,  69,  10, 112,  77,  60,
       122, 114,  17,  18, 126,  38, 107,  39, 111,  81,  42,  63,  46,
       125, 102,  78,  30,  21, 103,  49,  26,  24,  80, 113,  83,   6,
        56,  40,  98,  32,  70,  47, 110,  33, 104, 109,  45,  22,   2,
        14,  66,  23,  55,  65,  76, 124,  64,  74,  89,  87, 117, 123,
        28,  51,   1,  50,  62,  75, 106,  68,  36,  88, 108])

In [17]:
num_to_chr = {}
for i in data['cat3'].unique():
    num_to_chr[i] = label_encoder.inverse_transform(np.array([i]))[0]

In [18]:
label_encoder.inverse_transform(np.array([111]))

array(['테마공원'], dtype=object)

In [19]:
len(set(data['cat3'].tolist()))

128

In [20]:
data = data.rename(columns={'overview':'content', 'cat3':'label'})

In [21]:
data

,content,label
0,항구 포구에 대한 내용입니다. 소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이...,120
1,골프에 대한 내용입니다. 경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다....,8
2,한식에 대한 내용입니다. 금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는...,118
3,한식에 대한 내용입니다. 철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경...,118
4,한식에 대한 내용입니다. ※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아...,118
...,...,...
16981,한식에 대한 내용입니다. 해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤...,118
16982,모텔에 대한 내용입니다. 설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로...,31
16983,모텔에 대한 내용입니다. 충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는...,31
16984,야영장 오토캠핑장에 대한 내용입니다. 토토큰바위캠핑장은 경기도 가평지역 내에서도 청...,73


In [22]:
data_list=[]
for idx, row in tqdm(data.iterrows()):
    data_list.append([row[0], row[1]])


16986it [00:00, 23134.21it/s]


In [23]:
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [24]:
# 학습
# Train / Test set 분리
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

# 학습을 위한 데이터셋 구축
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)


In [25]:
######  tmp
# 학습 데이터셋 구축 클래스
class BERTDataset(Dataset):
    
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        print(transform)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        print(self.sentences)
        self.labels = [np.int32(i[label_idx]) for i in dataset]
        print(self.labels)

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
######  tmp

In [26]:
# from gluonnlp.data import SentencepieceTokenizer
# from kobert import get_tokenizer
# tok_path = get_tokenizer()
# sp  = SentencepieceTokenizer(tok_path)

In [27]:
# sp('5일장')

In [28]:

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

# warm_up
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

# 스케쥴러
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out, hidden = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))


    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out, hidden = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


    # GPU 사용시 학습 전 캐시제거
    gc.collect()
    torch.cuda.empty_cache()




1it [00:03,  3.50s/it]

epoch 1 batch id 1 loss 5.004242897033691 train acc 0.015625


201it [01:27,  2.37it/s]

epoch 1 batch id 201 loss 1.511103630065918 train acc 0.42428482587064675


213it [01:32,  2.29it/s]

...


54it [00:08,  6.35it/s]


epoch 1 test acc 0.9074074074074074


1it [00:00,  1.88it/s]

epoch 2 batch id 1 loss 1.4317032098770142 train acc 0.890625


32it [00:13,  2.36it/s]

In [ ]:
# 모델 로컬 저장
torch.save(model, f'./result_m.pt')
torch.save(model.state_dict(), f'./result_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, f'./result.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능


In [ ]:
# model load
model_loaded = torch.load(f'./result_m.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model_loaded.load_state_dict(torch.load(f'./result_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장
checkpoint = torch.load(f'./result.tar')   # dict 불러오기
model_loaded.load_state_dict(checkpoint['model'])


In [ ]:
# 예측값의 확률 값을 출력하기 위한 함수
def softmax(logits):
    argmax = torch.argmax(logits).item()
    sum_exp = torch.exp(logits).sum().item()
    argmax_exp = torch.exp(logits)[argmax].item()
    return argmax, argmax_exp/sum_exp, logits

In [ ]:
def bert_predict(predict_sentence, thres = 0.5):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model_loaded.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out, sentence_vector = model_loaded(token_ids, valid_length, segment_ids)
#         print(out, sentence_vector)


        for i in out:

            argmax, prob, logits = softmax(i)

    return argmax, prob, logits # sentence_vector,

In [ ]:
testset = pd.read_csv('./test.csv')

In [ ]:
test_sentences = testset['overview'].tolist()

In [ ]:
# before
pred = []
for i in tqdm(test_sentences):
    pred.append(num_to_chr[bert_predict(i)[0]])
    

In [ ]:

pred = []
for i in tqdm(test_sentences):
    pred.append(num_to_chr[bert_predict(i)[0]])
    

In [ ]:
testset['cat3'] = pred
testset[['id', 'cat3']].to_csv('kobert_add_label.csv', index=False)